# Motivation
I noticed that my baby was very interested in LED lights, so I want to make my CPX display its NeoPixel lights for him in a simple interactive way.  Detecting the distance of a hand is possible with a laser based distance sensor.

# Circuit
On the **VL6180X Time of Flight Mirco-LIDAR Distance Sensor Breakout** board connect to CPX:
- VIN -> 3.3V  (red)
- GND -> GND   (green)
- SCL -> SCL   (white, I2C bus Serial CLock)
- SDA -> SDA   (yellow, I2C bus Serial DAta)

<img src="circuit.jpg">

# Test the Library
Download the *precompiled* [adafruit_vl6180x.mpy](https://github.com/adafruit/Adafruit_CircuitPython_VL6180X/releases/download/1.0.1/adafruit-circuitpython-vl6180x-2.2.1-mpy-1.0.1.zip) module and copy to the CPX.

Test out the sensor with Adafruit's test code, copy to `code.py`:
```python
# Demo of reading the range and lux from the VL6180x distance sensor and
# printing it every second.
# Author: Tony DiCola
import time

import board
import busio

import adafruit_vl6180x


# Create I2C bus.
i2c = busio.I2C(board.SCL, board.SDA)

# Create sensor instance.
sensor = adafruit_vl6180x.VL6180X(i2c)

# Main loop prints the range and lux every second:
while True:
    # Read the range in millimeters and print it.
    range_mm = sensor.range
    print('Range: {0}mm'.format(range_mm))
    # Read the light, note this requires specifying a gain value:
    # - adafruit_vl6180x.ALS_GAIN_1 = 1x
    # - adafruit_vl6180x.ALS_GAIN_1_25 = 1.25x
    # - adafruit_vl6180x.ALS_GAIN_1_67 = 1.67x
    # - adafruit_vl6180x.ALS_GAIN_2_5 = 2.5x
    # - adafruit_vl6180x.ALS_GAIN_5 = 5x
    # - adafruit_vl6180x.ALS_GAIN_10 = 10x
    # - adafruit_vl6180x.ALS_GAIN_20 = 20x
    # - adafruit_vl6180x.ALS_GAIN_40 = 40x
    light_lux = sensor.read_lux(adafruit_vl6180x.ALS_GAIN_1)
    print('Light (1x gain): {0}lux'.format(light_lux))
    # Delay for a second.
    time.sleep(1.0)
```

Here is some sample output from the REPL:
```
Auto-reload is on. Simply save files over USB to run them or enter REPL to disable.
code.py output:

Range: 255mm
Light (1x gain): 7.04lux
Range: 255mm
Light (1x gain): 7.36lux
Range: 255mm
Light (1x gain): 7.36lux
Range: 44mm
Light (1x gain): 6.4lux
Range: 29mm
Light (1x gain): 4.48lux
Range: 45mm
Light (1x gain): 5.76lux
Range: 255mm
Light (1x gain): 5.44lux
Range: 255mm
Light (1x gain): 5.44lux
Range: 255mm
Light (1x gain): 5.76lux
Range: 132mm
Light (1x gain): 7.36lux
Range: 71mm
Light (1x gain): 7.04lux
Range: 47mm
Light (1x gain): 6.4lux
Range: 41mm
Light (1x gain): 6.08lux
Range: 52mm
Light (1x gain): 6.4lux
Range: 56mm
Light (1x gain): 6.08lux
```

# Complete the Project
Now we make a simple NeoPixel display to show how close we are to the LIDAR sensor.
```python
# Demo of reading the range from the VL6180x distance sensor and
# showing a "closeness" reading NeoPixel display.
# Author: Craig Versek, based on example and library by Tony DiCola
import time

import board
import busio

import neopixel
import adafruit_vl6180x

#setup the NeoPixels
pixels = neopixel.NeoPixel(board.NEOPIXEL, 10, brightness=.2)
pixels.fill((0,0,0))
pixels.show()
NUM_PIXELS = len(pixels)

# Create I2C bus.
i2c = busio.I2C(board.SCL, board.SDA)

# Create sensor instance.
sensor = adafruit_vl6180x.VL6180X(i2c)

# Main loop prints the range second and updates NeoPixel display:
while True:
    # Read the range in millimeters and print it.
    range_mm = sensor.range
    print((range_mm,))
    # Compute the number of pixels to light up based on range
    level = int(3*NUM_PIXELS*float(255 - range_mm)/255.0)
    # Display
    blue_pixels  = level - 2*NUM_PIXELS   #very close
    green_pixels = level - NUM_PIXELS     #medium close
    red_pixels   = level % NUM_PIXELS     #not so close
    if blue_pixels > 0:
        for i in range(blue_pixels):
            pixels[i] = (0,0,255)
    elif green_pixels > 0:
        for i in range(green_pixels):
            pixels[i] = (0,255,0)
    else:
        for i in range(red_pixels):
            pixels[i] = (255,0,0)    
        for i in range(level,NUM_PIXELS): #OFF pixels
            pixels[i] = (0,0,0)

```